In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torchvision.models import ResNet18_Weights

In [3]:
transform_train = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
])

transform_val = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Load dataset
train_data = datasets.ImageFolder('/kaggle/input/normalizedecgimagedata/Thesis3/train', transform=transform_train)
val_data = datasets.ImageFolder('/kaggle/input/normalizedecgimagedata/Thesis3/validation', transform=transform_val)
test_data = datasets.ImageFolder('/kaggle/input/normalizedecgimagedata/Thesis3/test', transform=transform_val)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = DataLoader(val_data, batch_size=64)
test_loader = DataLoader(test_data, batch_size=64)

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class ResNetWithDropout(nn.Module):
    def __init__(self, base_model, num_classes, dropout_rate=0.5):
        super().__init__()
        self.features = nn.Sequential(*list(base_model.children())[:-1])
        self.dropout = nn.Dropout(dropout_rate)
        self.classifier = nn.Linear(base_model.fc.in_features, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = self.classifier(x)
        return x

# Load model
from torchvision.models import resnet18, ResNet18_Weights
base_model = resnet18(weights=ResNet18_Weights.DEFAULT)
model = ResNetWithDropout(base_model, num_classes=len(train_data.classes), dropout_rate=0.5)
model = model.to(device)

In [6]:
import torch.optim as optim

num_epochs = 3

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)

for epoch in range(num_epochs):
    model.train()
    total_loss, total_correct = 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)

        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_correct += (outputs.argmax(1) == labels).sum().item()

    acc = 100 * total_correct / len(train_loader.dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {total_loss:.4f}, Accuracy: {acc:.2f}%")

    # Validation
    model.eval()
    val_correct = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            val_correct += (outputs.argmax(1) == labels).sum().item()
    val_acc = 100 * val_correct / len(val_loader.dataset)
    print(f"Validation Accuracy: {val_acc:.2f}%")

Epoch [1/3], Train Loss: 29.5760, Accuracy: 93.16%
Validation Accuracy: 94.50%
Epoch [2/3], Train Loss: 13.7549, Accuracy: 96.81%
Validation Accuracy: 96.15%
Epoch [3/3], Train Loss: 10.9033, Accuracy: 97.71%
Validation Accuracy: 97.52%


In [8]:
import torch.optim as optim

num_epochs = 4

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)

for epoch in range(num_epochs):
    model.train()
    total_loss, total_correct = 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)

        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_correct += (outputs.argmax(1) == labels).sum().item()

    acc = 100 * total_correct / len(train_loader.dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {total_loss:.4f}, Accuracy: {acc:.2f}%")

    # Validation
    model.eval()
    val_correct = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            val_correct += (outputs.argmax(1) == labels).sum().item()
    val_acc = 100 * val_correct / len(val_loader.dataset)
    print(f"Validation Accuracy: {val_acc:.2f}%")

Epoch [1/4], Train Loss: 5.2902, Accuracy: 99.04%
Validation Accuracy: 94.54%
Epoch [2/4], Train Loss: 4.3482, Accuracy: 99.16%
Validation Accuracy: 96.33%
Epoch [3/4], Train Loss: 2.9524, Accuracy: 99.35%
Validation Accuracy: 96.74%
Epoch [4/4], Train Loss: 2.5992, Accuracy: 99.57%
Validation Accuracy: 98.58%


In [9]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

Test Accuracy: 98.35%


In [10]:
torch.save(model, 'ecg_model.pth')